**Analítica de datos en salud**

---

***Tarea No. 2: Aplicación de Extracción de Información***

Presentado por:

* 2400452 - Jennifer Benavides Castillo
* 2400479 - Cristhian David Cruz Millán
* 2400794 - Sergio Alejandro Fierro Ospitia
* 2400478 - Edwin Andrés Lasso Rosero

#Entregable 3:
Integrar los modelos de extracción de entidades (NER) y clasificación de contexto (negación / afirmación / incertidumbre) en un único flujo de procesamiento, que lea historias clínicas y produzca una base de datos estructurada (CSV).

##Sirve para:

* Consolidar la información clínica en un formato analizable, con cuatro columnas clave:

 * patient_id: Identificador del paciente.

 * sentence: Oración donde se encontró la entidad.

 * NER: Entidad médica extraída.

 * Estado: Clasificación del contexto (Afirmada, Negada, Sospechosa).

###Esta base estructurada es útil para:

* Estudios médicos retrospectivos.

* Vigilancia de enfermedades.

* Entrenamiento de otros modelos clínicos.

* Tareas clínicas asistidas por IA.

In [ ]:
#IMPORTAR LIBRERIAS
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline, AutoConfig
from google.colab import files
import pandas as pd
import chardet

In [ ]:
#CONEXIÓN A DRIVE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Carga del archivo desde la ruta correcta resultado del entregable 1
INPUT_FILE = "/content/drive/MyDrive/entidades_extraidas.csv"

#Carga del modelo
MODEL_NEGATION = "JuanSolarte99/bert-base-uncased-finetuned-ner-negation_detection_NUBES"

DEVICE = 0  # o -1 si no hay GPU

In [ ]:
#Cargar modelo de negación
print("Cargando modelo de negación e incertidumbre...")
config = AutoConfig.from_pretrained(MODEL_NEGATION)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NEGATION)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NEGATION)

negation_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=DEVICE
)

Cargando modelo de negación e incertidumbre...


Device set to use cpu


In [ ]:
#Lectura de datos de entidades extraídas

print(f"Cargando archivo de entidades: {INPUT_FILE}")
df_entidades = pd.read_csv(INPUT_FILE, sep=';', encoding='latin-1', engine='python')


Cargando archivo de entidades: /content/drive/MyDrive/entidades_extraidas.csv


In [ ]:
#Filtrar valores vacios
df_entidades = df_entidades[df_entidades["sentence"].notna() & (df_entidades["sentence"].str.strip() != "")]

In [ ]:
#FUNCIÓN: Clasificación del estado contextual

def clasificar_estado(etiquetas_detectadas):
    if any(et in etiquetas_detectadas for et in ["LABEL_0", "LABEL_1"]):
        return "Negada"
    elif any(et in etiquetas_detectadas for et in ["LABEL_2", "LABEL_3", "LABEL_5", "LABEL_6"]):
        return "Sospechosa"
    else:
        return "Afirmada"

In [ ]:
#Define el modelo de contexto (negación/incertidumbre)
modelo_contexto = "JuanSolarte99/bert-base-uncased-finetuned-ner-negation_detection_NUBES"

#Cargar el modelo y pipeline
tokenizer_contexto = AutoTokenizer.from_pretrained(modelo_contexto)
modelo_contexto = AutoModelForTokenClassification.from_pretrained(modelo_contexto)

pipe_contexto = pipeline(
    "ner",
    model="JuanSolarte99/bert-base-uncased-finetuned-ner-negation_detection_NUBES",
    tokenizer="JuanSolarte99/bert-base-uncased-finetuned-ner-negation_detection_NUBES",
    aggregation_strategy="simple",
    device=0
)

Device set to use cpu


In [ ]:
#Clasificación del estado contextual de cada oración

estado_oraciones = {}
oraciones_unicas = df_entidades["sentence"].unique()

for oracion in oraciones_unicas:
    resultado = pipe_contexto(oracion)
    etiquetas = list(set([r['entity_group'] for r in resultado]))
    estado_oraciones[oracion] = clasificar_estado(etiquetas)

In [ ]:
#Asignar estado a cada fila del DataFrame

df_entidades["Estado"] = df_entidades["sentence"].map(estado_oraciones)

In [ ]:
#Seleccionar y renombrar columnas para la base final

df_final = df_entidades[["patient_id", "sentence", "entity", "Estado"]].copy()
df_final.rename(columns={"entity": "NER"}, inplace=True)

In [ ]:
#Exportar CSV con resultados
df_final.to_csv("base_datos_estructurada.csv", index=False)

print("Proceso completado. Archivo generado: base_datos_estructurada.csv")
df_final.head()

Proceso completado. Archivo generado: base_datos_estructurada.csv


,patient_id,sentence,NER,Estado
0,1126737,Mujer de 44 años diagnosticada de Carcinoma du...,44 años,Afirmada
1,1126737,Mujer de 44 años diagnosticada de Carcinoma du...,diagnosticada,Afirmada
2,1126737,Mujer de 44 años diagnosticada de Carcinoma du...,Carcinoma ductal infiltrante de mama derecha,Afirmada
3,1126737,Mujer de 44 años diagnosticada de Carcinoma du...,cT2N0M0,Afirmada
4,1126737,Mujer de 44 años diagnosticada de Carcinoma du...,estadio IIA,Afirmada


In [ ]:
#Guardar el DataFrame final en un archivo CSV
df_entidades.to_csv("base_datos_final.csv", index=False)

files.download("base_datos_final.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>